# DE ESTOS NO SIRVE NINGUNO, VAYANSE A TOMAR POR 🍑

# EJEMPLO 1 'POBLACION"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping

# Cargar el archivo con 10 años de datos
file_path = "cuadrantes_df_negocios_mes_anio.xlsx"
df_negocios = pd.read_excel(file_path)

# Asumo que la columna de población es la segunda (índice 1) y la renombro.
df_negocios.rename(columns={df_negocios.columns[1]: 'POBLACION'}, inplace=True)

# --- 1. Desapilar (Unpivot) ---
# Se mantiene 'CUADRANTE' y 'POBLACION' como identificadores fijos.
id_vars = ['CUADRANTE', 'POBLACION']
df_long = df_negocios.melt(
    id_vars=id_vars,
    var_name='MES_ANIO',
    value_name='ROBOS_MES_N'
)

# --- 2. Extraer AÑO y MES_N ---
# Asumo que el formato sigue siendo 'ROBOS A NEGOCIOS MES M YYYY'
df_long[['BASURA', 'BASURA2', 'BASURA3', 'BASURA4', 'MES_N', 'ANIO']] = \
    df_long['MES_ANIO'].str.split(' ', expand=True)

df_long.drop(columns=['MES_ANIO', 'BASURA', 'BASURA2', 'BASURA3', 'BASURA4'], inplace=True)

df_long['MES_N'] = pd.to_numeric(df_long['MES_N'])
df_long['ANIO'] = pd.to_numeric(df_long['ANIO'])

# --- 3. Ordenar y Crear Indexador Único ---
df_long.sort_values(by=['ANIO', 'MES_N', 'CUADRANTE'], inplace=True)
df_long.reset_index(drop=True, inplace=True)

# 4. Crear las variables N-1 y N-2 agrupadas por cuadrante
df_long['ROBOS_MES_N_MENOS_1'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
df_long['ROBOS_MES_N_MENOS_2'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# --- 5. Imputar el Inicio de la Serie (Ene 2015, Feb 2015) ---
# Imputamos los valores NaN iniciales con 0
df_long.fillna(0, inplace=True)

df_final = df_long[['CUADRANTE', 'POBLACION', 'ANIO', 'MES_N', 
                    'ROBOS_MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]

print("### 1. DataFrame Final (Formato de Serie de Tiempo con Población) ###")
print(f"Total de filas: {len(df_final)}") # Debería ser 9360
print(df_final.head(5))
print("-" * 50)

In [ ]:
# --- 1. Separar y Escalar los Datos ---
# Variables de Entrada (X): CUADRANTE, POBLACION, ANIO, MES_N, N-1, N-2 (6 variables)
X_scale = df_final[['CUADRANTE', 'POBLACION', 'ANIO', 'MES_N', 
                    'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].copy()
Y_scale = df_final[['ROBOS_MES_N']].copy()

# Entrenar los escaladores
scaler_X = MinMaxScaler()
X_scaled_values = scaler_X.fit_transform(X_scale)
X_scaled = pd.DataFrame(X_scaled_values, columns=X_scale.columns)

scaler_Y = MinMaxScaler()
Y_scaled_values = scaler_Y.fit_transform(Y_scale)
Y_scaled = pd.DataFrame(Y_scaled_values, columns=Y_scale.columns)


# --- 2. Agrupar los Datos Escaldados (120 Bloques) ---
# Usamos el índice compuesto (AÑO y MES_N) para agrupar
df_scaled_grouped = X_scaled.copy()
df_scaled_grouped['ROBOS_MES_N'] = Y_scaled['ROBOS_MES_N']
df_scaled_grouped['GRUPO_MES_ANIO'] = df_final['ANIO'].astype(str) + '-' + df_final['MES_N'].astype(str)


# X: Matriz (120 filas, 78 cuadrantes * 6 entradas = 468 columnas)
X_cols = ['CUADRANTE', 'POBLACION', 'ANIO', 'MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']
X_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')[X_cols].apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data_final)

# Y: Matriz (120 filas, 78 columnas)
Y_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')['ROBOS_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data_final)

# Dimensiones
input_dim = X.shape[1] # 468
output_dim = Y.shape[1] # 78

# 3. División para Entrenamiento/Prueba (15 meses para prueba)
test_size_months = 15
X_train, X_test, Y_train, Y_test = X[:-test_size_months], X[-test_size_months:], Y[:-test_size_months], Y[-test_size_months:]

print(f"Dimensión de X (Entrenamiento): {X_train.shape}")
print(f"Dimensión de X (Prueba): {X_test.shape}")
print(f"Nueva Dimensión de Entrada: {input_dim} (78 cuadrantes * 6 variables)")
print("-" * 50)

In [ ]:
# --- 1. Construir y Compilar el Modelo ---
# La capa de entrada se ajusta automáticamente a input_dim = 468
model = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim, activation='linear') # 78 salidas
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

print("### 3. Entrenamiento del Modelo Keras (6 Entradas, 10 Años) ###")
history = model.fit(
    X_train, Y_train,
    epochs=300,
    batch_size=4,
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0
)
print("Entrenamiento finalizado. El modelo ahora incluye la Población como factor.")
print("-" * 50)

# --- 4. Evaluación y R^2 ---
Y_pred_test = model.predict(X_test)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled = scaler_Y.inverse_transform(Y_pred_test.reshape(-1, 1))

r2_final = r2_score(Y_test_descaled, Y_pred_descaled)

print(f"### 4. Coeficiente de Determinación (R^2) Final: {r2_final:.4f} ###")
print("-" * 50)

# EJEMPLO 2 abajo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping

# Cargar el archivo con 10 años de datos
file_path = "cuadrantes_df_negocios_mes_anio.xlsx"
df_negocios = pd.read_excel(file_path)

# --- 1. Desapilar (Unpivot) ---
# Transformar la tabla ancha a larga
df_long = df_negocios.melt(
    id_vars=['CUADRANTE'],
    var_name='MES_ANIO',
    value_name='ROBOS_MES_N'
)

# --- 2. Extraer AÑO y MES_N ---
# La columna MES_ANIO tiene el formato 'ROBOS A NEGOCIOS MES M YYYY'
df_long[['BASURA', 'BASURA2', 'BASURA3', 'BASURA4', 'MES_N', 'ANIO']] = \
    df_long['MES_ANIO'].str.split(' ', expand=True)

df_long.drop(columns=['MES_ANIO', 'BASURA', 'BASURA2', 'BASURA3', 'BASURA4'], inplace=True)

df_long['MES_N'] = pd.to_numeric(df_long['MES_N'])
df_long['ANIO'] = pd.to_numeric(df_long['ANIO'])

# --- 3. Ordenar y Crear Indexador Único (Para la serie de tiempo) ---
df_long.sort_values(by=['ANIO', 'MES_N', 'CUADRANTE'], inplace=True)
df_long.reset_index(drop=True, inplace=True)

# 4. Crear las variables N-1 y N-2 agrupadas por cuadrante
# Esto maneja automáticamente las transiciones de mes y año dentro de cada cuadrante
df_long['ROBOS_MES_N_MENOS_1'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
df_long['ROBOS_MES_N_MENOS_2'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# --- 5. Imputar el Inicio de la Serie (Ene 2015, Feb 2015) ---
# Los primeros 78*2 = 156 robos tienen NaN en N-1 y N-2.
# Imputamos con 0, ya que no tenemos datos reales anteriores a Ene 2015.
# (Alternativamente, se podría usar el promedio histórico si se tuviera).
df_long.fillna(0, inplace=True)

df_final = df_long[['CUADRANTE', 'ANIO', 'MES_N', 'ROBOS_MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]

print("### 1. DataFrame Final (Formato de Serie de Tiempo) ###")
print(f"Total de filas: {len(df_final)}") # Debería ser 9360
print(df_final.head(5))
print("-" * 50)

In [ ]:
# exportar a excel
# df_final.to_excel('df_finalnegocio_serie_tiempo.xlsx', index=False)

In [ ]:
# --- 1. Separar y Escalar los Datos ---
# Variables de Entrada (X): CUADRANTE, ANIO, MES_N, N-1, N-2
# Variable de Salida (Y): ROBOS_MES_N
X_scale = df_final[['CUADRANTE', 'ANIO', 'MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].copy()
Y_scale = df_final[['ROBOS_MES_N']].copy()

# Entrenar el escalador solo con X (y Y)
scaler_X = MinMaxScaler()
X_scaled_values = scaler_X.fit_transform(X_scale)
X_scaled = pd.DataFrame(X_scaled_values, columns=X_scale.columns)

scaler_Y = MinMaxScaler()
Y_scaled_values = scaler_Y.fit_transform(Y_scale)
Y_scaled = pd.DataFrame(Y_scaled_values, columns=Y_scale.columns)


# --- 2. Agrupar los Datos Escaldados (Creando los 120 Bloques) ---
# Usamos un índice compuesto (AÑO y MES_N) para agrupar
df_scaled_grouped = X_scaled.copy()
df_scaled_grouped['ROBOS_MES_N'] = Y_scaled['ROBOS_MES_N']
df_scaled_grouped['GRUPO_MES_ANIO'] = df_final['ANIO'].astype(str) + '-' + df_final['MES_N'].astype(str)


# X: Matriz (120 filas, 78*5=390 columnas)
X_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')[['CUADRANTE', 'ANIO', 'MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data_final)

# Y: Matriz (120 filas, 78 columnas)
Y_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')['ROBOS_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data_final)

# Dimensiones
input_dim = X.shape[1] # 78 cuadrantes * 5 entradas = 390
output_dim = Y.shape[1] # 78

# 3. División para Entrenamiento/Prueba (Usamos los últimos 15 meses para prueba)
test_size_months = 15 # 15 meses para prueba
X_train, X_test, Y_train, Y_test = X[:-test_size_months], X[-test_size_months:], Y[:-test_size_months], Y[-test_size_months:]

print(f"Dimensión de X (Entrenamiento): {X_train.shape}") # Debería ser ~105 filas
print(f"Dimensión de X (Prueba): {X_test.shape}") # Debería ser 15 filas
print("-" * 50)

In [ ]:
# ASUMIMOS QUE X_train, Y_train, input_dim, output_dim, y los scalers ESTÁN DEFINIDOS

from tensorflow.keras.layers import LSTM # <-- Nueva capa clave

# Adaptar las matrices X para la capa LSTM (Añadir un timestep = 1)
# Forma original: (Muestras, Características) -> (105, 390)
# Nueva forma requerida por LSTM: (Muestras, Timesteps, Características)
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

print(f"Nueva Dimensión de X_train para LSTM: {X_train_lstm.shape}")
print("-" * 50)


# --- 1. Construir el Modelo LSTM ---
model_lstm = Sequential([
    # Capa LSTM: Ideal para la secuencia temporal. return_sequences=False porque
    # solo tenemos 1 timestep.
    LSTM(128, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    
    Dropout(0.3),
    
    # Capa Densa (para integrar los resultados de LSTM)
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Capa de salida (78 salidas)
    Dense(output_dim, activation='linear')
])

model_lstm.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

early_stop_lstm = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

print("### Entrenamiento del Modelo LSTM (Con 10 Años de Datos) ###")
history_lstm = model_lstm.fit(
    X_train_lstm, Y_train,
    epochs=500, # Aumentamos las épocas
    batch_size=4,
    validation_data=(X_test_lstm, Y_test),
    callbacks=[early_stop_lstm],
    verbose=0
)
print("Entrenamiento LSTM finalizado.")
print("-" * 50)

# --- 2. Evaluación y R^2 ---
Y_pred_test_lstm = model_lstm.predict(X_test_lstm)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled_lstm = scaler_Y.inverse_transform(Y_pred_test_lstm.reshape(-1, 1))

r2_lstm_final = r2_score(Y_test_descaled, Y_pred_descaled_lstm)

print(f"### Nuevo Coeficiente de Determinación (R^2) con LSTM: {r2_lstm_final:.4f} ###")

In [ ]:
# --- 1. Construir y Compilar el Modelo ---
model = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim, activation='linear') # 78 salidas
])

# Usamos un Learning Rate conservador para evitar divergencia con más datos
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

print("### 3. Entrenamiento del Modelo Keras (10 Años de Datos) ###")
history = model.fit(
    X_train, Y_train,
    epochs=300, # Aumentamos las épocas
    batch_size=4,
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0
)
print("Entrenamiento finalizado. El modelo es mucho más robusto ahora.")
print("-" * 50)

# --- 4. Evaluación y R^2 ---
Y_pred_test = model.predict(X_test)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled = scaler_Y.inverse_transform(Y_pred_test.reshape(-1, 1))

r2_final = r2_score(Y_test_descaled, Y_pred_descaled)

print(f"### 4. Coeficiente de Determinación (R^2) Final: {r2_final:.4f} ###")

if r2_final > 0.6:
    print("✅ ¡El modelo ha mejorado significativamente su precisión con más datos!")
elif r2_final > 0.0:
    print("⚠️ Precisión positiva. Se puede mejorar ajustando hiperparámetros.")
else:
    print("❌ Aún hay problemas. Revisar el escalado o la arquitectura.")
print("-" * 50)

# Ejemplo 3 con POBLACION, sin MES_N y ANIO, y CUADRANTE en variable DUMMIE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping

# Cargar el archivo con 10 años de datos
file_path = "cuadrantes_df_negocios_mes_anio.xlsx"
df_wide = pd.read_excel(file_path)

# Asumo que la columna de población es la segunda y la renombro.
df_wide.rename(columns={df_wide.columns[1]: 'POBLACION'}, inplace=True)

# --- 1. Desapilar (Unpivot) ---
id_vars = ['CUADRANTE', 'POBLACION']
df_long = df_wide.melt(
    id_vars=id_vars,
    var_name='MES_ANIO',
    value_name='ROBOS_MES_N'
)

# --- 2. Extraer AÑO y MES_N ---
df_long[['BASURA', 'BASURA2', 'BASURA3', 'BASURA4', 'MES_N', 'ANIO']] = \
    df_long['MES_ANIO'].str.split(' ', expand=True)

df_long.drop(columns=['MES_ANIO', 'BASURA', 'BASURA2', 'BASURA3', 'BASURA4'], inplace=True)

df_long['MES_N'] = pd.to_numeric(df_long['MES_N'])
df_long['ANIO'] = pd.to_numeric(df_long['ANIO'])

# --- 3. Ordenar y Crear Indexador Único ---
df_long.sort_values(by=['ANIO', 'MES_N', 'CUADRANTE'], inplace=True)
df_long.reset_index(drop=True, inplace=True)

# 4. Crear las variables N-1 y N-2
#df_long['ROBOS_MES_N_MENOS_1'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(1)
#df_long['ROBOS_MES_N_MENOS_2'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(2)

# --- 5. Imputar el Inicio de la Serie ---
#df_long.fillna(0, inplace=True)

#df_final = df_long[['CUADRANTE', 'POBLACION', 'ANIO', 'MES_N', 
#                    'ROBOS_MES_N', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']]

#print("### 1. DataFrame Final listo (9360 Filas) ###")
#print(df_final.head(5))
#print("-" * 50)

In [ ]:
# Asumimos que df_long del Paso 1 está cargado y ordenado por ANIO, MES_N, CUADRANTE.
# Si no lo tienes, usa el código completo del Paso 1 de la respuesta anterior para obtener df_long.

# 4. Crear las variables N-1 hasta N-6 agrupadas por cuadrante
for i in range(1, 7):
    df_long[f'ROBOS_MES_N_MENOS_{i}'] = df_long.groupby('CUADRANTE')['ROBOS_MES_N'].shift(i)

# --- 5. Imputar el Inicio de la Serie ---
# Ahora hay más NaNs al inicio (los primeros 78*6 = 468 valores).
df_long.fillna(0, inplace=True)

df_final = df_long[[
    'CUADRANTE', 'POBLACION', 'ANIO', 'MES_N', 'ROBOS_MES_N', 
    'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2', 'ROBOS_MES_N_MENOS_3',
    'ROBOS_MES_N_MENOS_4', 'ROBOS_MES_N_MENOS_5', 'ROBOS_MES_N_MENOS_6'
]].copy()

print("### 1. DataFrame Final con 6 Meses de Historia ###")
print(f"Total de filas: {len(df_final)}")
print(df_final.head(7))
print("-" * 50)

In [ ]:
# --- 1. Separar y Aplicar One-Hot Encoding (OHE) a CUADRANTE ---
df_final_ohe = df_final.copy()
df_ohe = pd.get_dummies(df_final_ohe['CUADRANTE'], prefix='C')
df_final_ohe = pd.concat([df_final_ohe.drop('CUADRANTE', axis=1), df_ohe], axis=1)

# --- 2. Separar y Escalar los Datos (9 Variables) ---
X_cols_ohe = [col for col in df_final_ohe.columns if col.startswith('C_')] # 78 columnas OHE

# Nuevas variables de entrada: 78 (OHE) + POBLACION + 6 Históricas
X_cols = X_cols_ohe + ['POBLACION']
X_cols += [f'ROBOS_MES_N_MENOS_{i}' for i in range(1, 7)] # 6 columnas de historia

X_scale = df_final_ohe[X_cols].copy()
Y_scale = df_final_ohe[['ROBOS_MES_N']].copy()

# Entrenar los escaladores (scaler_X y scaler_Y)
scaler_X = MinMaxScaler()
X_scaled_values = scaler_X.fit_transform(X_scale)
X_scaled = pd.DataFrame(X_scaled_values, columns=X_scale.columns)
scaler_Y = MinMaxScaler()
Y_scaled_values = scaler_Y.fit_transform(Y_scale)
Y_scaled = pd.DataFrame(Y_scaled_values, columns=Y_scale.columns)

# --- 3. Agrupar los Datos Escaldados (120 Bloques) ---
df_scaled_grouped = X_scaled.copy()
df_scaled_grouped['ROBOS_MES_N'] = Y_scaled['ROBOS_MES_N']
df_scaled_grouped['GRUPO_MES_ANIO'] = df_final_ohe['ANIO'].astype(str) + '-' + df_final_ohe['MES_N'].astype(str)

# X: Matriz (120 filas, 78 cuadrantes * 85 entradas = 6630 columnas)
X_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')[X_cols].apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data_final)

# Y: Matriz (120 filas, 78 columnas)
Y_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')['ROBOS_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data_final)

# Dimensiones
input_dim = X.shape[1] # 6630
output_dim = Y.shape[1] # 78

# División para Entrenamiento/Prueba (15 meses para prueba)
test_size_months = 15
X_train, X_test, Y_train, Y_test = X[:-test_size_months], X[-test_size_months:], Y[:-test_size_months], Y[-test_size_months:]

print(f"Nueva Dimensión de Entrada: {input_dim} (78 cuadrantes * 85 variables)")
print("-" * 50)

In [ ]:
# --- 1. Construir y Compilar el Modelo ---
model = Sequential([
    Dense(1024, activation='relu', input_shape=(input_dim,)), 
    Dropout(0.3),
    Dense(512, activation='relu'), 
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

print("### 3. Entrenamiento con 6 Meses de Historia ###")
history = model.fit(
    X_train, Y_train,
    epochs=500, 
    batch_size=2, 
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0
)
print("Entrenamiento finalizado. El modelo ahora tiene un contexto de 6 meses.")
print("-" * 50)

# --- 4. Evaluación y R^2 ---
Y_pred_test = model.predict(X_test)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled = scaler_Y.inverse_transform(Y_pred_test.reshape(-1, 1))

r2_final = r2_score(Y_test_descaled, Y_pred_descaled)

print(f"### 4. Coeficiente de Determinación (R^2) Final (¡Busca el 70%+!): {r2_final:.4f} ###")
print("-" * 50)

In [ ]:
# --- 1. Separar y Aplicar One-Hot Encoding (OHE) a CUADRANTE ---
df_final_ohe = df_final.copy()

# Aplicar OHE a la columna CUADRANTE
df_ohe = pd.get_dummies(df_final_ohe['CUADRANTE'], prefix='C')
df_final_ohe = pd.concat([df_final_ohe.drop('CUADRANTE', axis=1), df_ohe], axis=1)


# --- 2. Separar y Escalar los Datos (Nuevas Entradas) ---
# X_cols: 78 columnas OHE + POBLACION + N-1 + N-2 (81 variables)
X_cols = [col for col in df_final_ohe.columns if col.startswith('C_')] # Las 78 columnas OHE
X_cols += ['POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']

X_scale = df_final_ohe[X_cols].copy()
Y_scale = df_final_ohe[['ROBOS_MES_N']].copy()

# Entrenar los escaladores
scaler_X = MinMaxScaler()
X_scaled_values = scaler_X.fit_transform(X_scale)
X_scaled = pd.DataFrame(X_scaled_values, columns=X_scale.columns)

scaler_Y = MinMaxScaler()
Y_scaled_values = scaler_Y.fit_transform(Y_scale)
Y_scaled = pd.DataFrame(Y_scaled_values, columns=Y_scale.columns)


# --- 3. Agrupar los Datos Escaldados (120 Bloques) ---
df_scaled_grouped = X_scaled.copy()
df_scaled_grouped['ROBOS_MES_N'] = Y_scaled['ROBOS_MES_N']
df_scaled_grouped['GRUPO_MES_ANIO'] = df_final_ohe['ANIO'].astype(str) + '-' + df_final_ohe['MES_N'].astype(str)

# X: Matriz (120 filas, 78 cuadrantes * 81 entradas = 6318 columnas)
X_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')[X_cols].apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data_final)

# Y: Matriz (120 filas, 78 columnas)
Y_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')['ROBOS_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data_final)

# Dimensiones
input_dim = X.shape[1] # 6318
output_dim = Y.shape[1] # 78

# División para Entrenamiento/Prueba (15 meses para prueba)
test_size_months = 15
X_train, X_test, Y_train, Y_test = X[:-test_size_months], X[-test_size_months:], Y[:-test_size_months], Y[-test_size_months:]

print(f"Dimensión de X (Entrenamiento): {X_train.shape}")
print(f"Nueva Dimensión de Entrada: {input_dim} (78 cuadrantes * 81 variables)")
print("-" * 50)

In [ ]:
# --- 1. Construir y Compilar el Modelo (Capacidad Aumentada) ---
model = Sequential([
    Dense(1024, activation='relu', input_shape=(input_dim,)), # Aumentada
    Dropout(0.3),
    Dense(512, activation='relu'), # Aumentada
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim, activation='linear') # 78 salidas
])

# Mantenemos el LR muy bajo para ser conservadores
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

print("### 3. Entrenamiento con One-Hot Encoding (OHE) y Mayor Capacidad ###")
history = model.fit(
    X_train, Y_train,
    epochs=500, 
    batch_size=2, 
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0
)
print("Entrenamiento finalizado. Este modelo aprende la firma de cada cuadrante.")
print("-" * 50)

# --- 4. Evaluación y R^2 ---
Y_pred_test = model.predict(X_test)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled = scaler_Y.inverse_transform(Y_pred_test.reshape(-1, 1))

r2_final = r2_score(Y_test_descaled, Y_pred_descaled)

print(f"### 4. Coeficiente de Determinación (R^2) Final (Objetivo 70%+): {r2_final:.4f} ###")
print("-" * 50)

In [ ]:
# --- 1. Separar y Escalar los Datos ---
# X AHORA SOLO CONTIENE 4 VARIABLES: CUADRANTE, POBLACION, N-1, N-2
X_scale = df_final[['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']].copy()
Y_scale = df_final[['ROBOS_MES_N']].copy()

# Entrenar los escaladores
scaler_X = MinMaxScaler()
X_scaled_values = scaler_X.fit_transform(X_scale)
X_scaled = pd.DataFrame(X_scaled_values, columns=X_scale.columns)

scaler_Y = MinMaxScaler()
Y_scaled_values = scaler_Y.fit_transform(Y_scale)
Y_scaled = pd.DataFrame(Y_scaled_values, columns=Y_scale.columns)


# --- 2. Agrupar los Datos Escaldados (120 Bloques) ---
df_scaled_grouped = X_scaled.copy()
df_scaled_grouped['ROBOS_MES_N'] = Y_scaled['ROBOS_MES_N']
# Usamos el índice de tiempo (AÑO y MES_N) solo para agrupar, pero NO como input de X
df_scaled_grouped['GRUPO_MES_ANIO'] = df_final['ANIO'].astype(str) + '-' + df_final['MES_N'].astype(str)


# X: Matriz (120 filas, 78 cuadrantes * 4 entradas = 312 columnas)
X_cols = ['CUADRANTE', 'POBLACION', 'ROBOS_MES_N_MENOS_1', 'ROBOS_MES_N_MENOS_2']
X_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')[X_cols].apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data_final)

# Y: Matriz (120 filas, 78 columnas)
Y_data_final = df_scaled_grouped.groupby('GRUPO_MES_ANIO')['ROBOS_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data_final)

# Dimensiones
input_dim = X.shape[1] # 312
output_dim = Y.shape[1] # 78

# 3. División para Entrenamiento/Prueba (15 meses para prueba)
test_size_months = 15
X_train, X_test, Y_train, Y_test = X[:-test_size_months], X[-test_size_months:], Y[:-test_size_months], Y[-test_size_months:]

print(f"Dimensión de X (Entrenamiento): {X_train.shape}")
print(f"Dimensión de X (Prueba): {X_test.shape}")
print(f"Nueva Dimensión de Entrada: {input_dim} (78 cuadrantes * 4 variables)")
print("-" * 50)

In [ ]:
# --- 1. Construir y Compilar el Modelo ---
# La capa de entrada se ajusta automáticamente a input_dim = 312
model = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(output_dim, activation='linear') # 78 salidas
])

# Usamos Learning Rate muy bajo y batch_size reducido para mejor convergencia
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

# Usamos alta paciencia
early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

print("### 3. Entrenamiento con 4 Entradas (Mejorado) ###")
history = model.fit(
    X_train, Y_train,
    epochs=500, 
    batch_size=2, 
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0
)
print("Entrenamiento finalizado. El modelo ahora enfoca su aprendizaje en las variables clave.")
print("-" * 50)

# --- 4. Evaluación y R^2 ---
Y_pred_test = model.predict(X_test)

# Desescalar la predicción para R^2 (usando el scaler_Y)
Y_test_descaled = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
Y_pred_descaled = scaler_Y.inverse_transform(Y_pred_test.reshape(-1, 1))

r2_final = r2_score(Y_test_descaled, Y_pred_descaled)

print(f"### 4. Coeficiente de Determinación (R^2) Final: {r2_final:.4f} ###")
print("-" * 50)